<h2 align="center">Employee Bonus Prediction</h2>

#### Load the Dataset

In [1]:
import pandas as pd

df = pd.read_csv('bonus.csv')
df

,employee_id,performance,years_of_experience,projects_completed,bonus
0,EMP_001,7,1,3,116
1,EMP_002,4,8,10,136
2,EMP_003,8,4,5,150
3,EMP_004,5,4,7,118
4,EMP_005,7,5,6,146
...,...,...,...,...,...
995,EMP_996,10,3,5,168
996,EMP_997,10,7,9,200
997,EMP_998,8,5,7,160
998,EMP_999,2,2,5,66


#### Train Test Split

In [17]:
from sklearn.model_selection import train_test_split

X = df[['performance','years_of_experience','projects_completed']].values
y = df[['bonus']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

#### Train a Neural Network

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0) #Every random number PyTorch generates will be the same each time you run the code

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [27]:
class BonusPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(3,1))
        
    def forward(self, x):
        return self.network(x)

In [31]:
model = BonusPredictor()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.005)

### Train Model

In [34]:
epochs = 5000

for epoch in range(epochs):
    # Forward pass
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()

    # Update parameters
    optimizer.step()

    # Print loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.2f}")


Epoch [100/5000], Loss: 20.23
Epoch [200/5000], Loss: 17.62
Epoch [300/5000], Loss: 15.43
Epoch [400/5000], Loss: 13.55
Epoch [500/5000], Loss: 11.91
Epoch [600/5000], Loss: 10.48
Epoch [700/5000], Loss: 9.21
Epoch [800/5000], Loss: 8.11
Epoch [900/5000], Loss: 7.13
Epoch [1000/5000], Loss: 6.27
Epoch [1100/5000], Loss: 5.52
Epoch [1200/5000], Loss: 4.85
Epoch [1300/5000], Loss: 4.27
Epoch [1400/5000], Loss: 3.76
Epoch [1500/5000], Loss: 3.30
Epoch [1600/5000], Loss: 2.91
Epoch [1700/5000], Loss: 2.56
Epoch [1800/5000], Loss: 2.25
Epoch [1900/5000], Loss: 1.98
Epoch [2000/5000], Loss: 1.74
Epoch [2100/5000], Loss: 1.53
Epoch [2200/5000], Loss: 1.35
Epoch [2300/5000], Loss: 1.18
Epoch [2400/5000], Loss: 1.04
Epoch [2500/5000], Loss: 0.92
Epoch [2600/5000], Loss: 0.81
Epoch [2700/5000], Loss: 0.71
Epoch [2800/5000], Loss: 0.62
Epoch [2900/5000], Loss: 0.55
Epoch [3000/5000], Loss: 0.48
Epoch [3100/5000], Loss: 0.42
Epoch [3200/5000], Loss: 0.37
Epoch [3300/5000], Loss: 0.33
Epoch [3400/5

In [38]:
# Switch to evaluation mode (important for models with dropout/batch norm layers)
model.eval()

with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)

print(f"Test Loss: {test_loss.item():.4f}")

Test Loss: 0.0360


In [42]:
X_test_tensor[:4]

tensor([[ 5.,  1.,  3.],
        [ 8.,  1.,  4.],
        [ 4.,  2.,  3.],
        [ 5., 10., 13.]])

In [46]:
test_predictions[:4]

tensor([[ 91.8145],
        [130.0862],
        [ 85.6353],
        [166.2442]])

In [48]:
y_test_tensor[:4]

tensor([[ 92.],
        [130.],
        [ 86.],
        [166.]])

In [50]:
for name, value in model.named_parameters():
    print(f"Name: {name}, Value: {value}")

Name: network.0.weight, Value: Parameter containing:
tensor([[12.0321,  5.8529,  2.1753]], requires_grad=True)
Name: network.0.bias, Value: Parameter containing:
tensor([19.2750], requires_grad=True)
